# Style Transfer

<img src="https://i0.wp.com/chelseatroy.com/wp-content/uploads/2018/12/neural_style_transfer.png?resize=768%2C311&ssl=1">

La idea de este trabajo final es reproducir el siguiente paper:

https://arxiv.org/pdf/1508.06576.pdf

El objetivo es transferir el estilo de una imagen dada a otra imagen distinta. 

Como hemos visto en clase, las primeras capas de una red convolucional se activan ante la presencia de ciertos patrones vinculados a detalles muy pequeños.

A medida que avanzamos en las distintas capas de una red neuronal convolucional, los filtros se van activando a medida que detectan patrones de formas cada vez mas complejos.

Lo que propone este paper es asignarle a la activación de las primeras capas de una red neuronal convolucional (por ejemplo VGG19) la definición del estilo y a la activación de las últimas capas de la red neuronal convolucional, la definición del contenido.

La idea de este paper es, a partir de dos imágenes (una que aporte el estilo y otra que aporte el contenido) analizar cómo es la activación de las primeras capas para la imagen que aporta el estilo y cómo es la activación de las últimas capas de la red convolucional para la imagen que aporta el contenido. A partir de esto se intentará sintetizar una imagen que active los filtros de las primeras capas que se activaron con la imagen que aporta el estilo y los filtros de las últimas capas que se activaron con la imagen que aporta el contenido.

A este procedimiento se lo denomina neural style transfer.

# En este trabajo se deberá leer el paper mencionado y en base a ello, entender la implementación que se muestra a continuación y contestar preguntas sobre la misma.

# Una metodología posible es hacer una lectura rápida del paper (aunque esto signifique no entender algunos detalles del mismo) y luego ir analizando el código y respondiendo las preguntas. A medida que se planteen las preguntas, volviendo a leer secciones específicas del paper terminará de entender los detalles que pudieran haber quedado pendientes.

Lo primero que haremos es cargar dos imágenes, una que aporte el estilo y otra que aporte el contenido. A tal fin utilizaremos imágenes disponibles en la web.

In [1]:

# Creamos el directorio para los archivos de salida
# !mkdir -p content/output

In [2]:
# Imagen para estilo
#!wget https://upload.wikimedia.org/wikipedia/commons/5/52/La_noche_estrellada1.jpg

# Imagen para contenido
#!wget https://upload.wikimedia.org/wikipedia/commons/thumb/f/f4/Neckarfront_T%C3%BCbingen_Mai_2017.jpg/775px-Neckarfront_T%C3%BCbingen_Mai_2017.jpg


In [3]:
# !mv *.jpg content/

In [4]:
from keras.preprocessing.image import load_img, save_img, img_to_array
import numpy as np
from scipy.optimize import fmin_l_bfgs_b
import time

from keras.applications import vgg19
from keras import backend as K
from pathlib import Path

In [5]:
# Habilita compatiblidad hacia atrás
import tensorflow._api.v2.compat.v1 as tf
tf.compat.v1.disable_eager_execution()

In [6]:
# Listo versiones de las librerias para futura referencia
import sys

import pandas as pd
import sklearn as sk
import tensorflow as tf
import keras
import platform

print(f"Python Platform: {platform.platform()}")
print(f"Tensor Flow Version: {tf.__version__}")
print(f"Keras Version: {keras.__version__}")
print()
print(f"Python {sys.version}")
print(f"Pandas {pd.__version__}")
print(f"Scikit-Learn {sk.__version__}")
gpu = len(tf.config.list_physical_devices('GPU'))>0
print("GPU is", "available" if gpu else "NOT AVAILABLE")

Python Platform: macOS-13.5.1-arm64-arm-64bit
Tensor Flow Version: 2.13.0
Keras Version: 2.13.1

Python 3.10.12 | packaged by conda-forge | (main, Jun 23 2023, 22:41:52) [Clang 15.0.7 ]
Pandas 2.0.3
Scikit-Learn 1.3.0
GPU is available


In [7]:
# Definimos las imagenes que vamos a utilizar, y el directorio de salida

base_image_path = Path.cwd() / Path("content/775px-Neckarfront_Tübingen_Mai_2017.jpg")
style_reference_image_path = Path.cwd() / Path("content/La_noche_estrellada1.jpg")
result_prefix = Path.cwd() / Path("content/output")
iterations = 100

In [8]:
base_image_path.name

'775px-Neckarfront_Tübingen_Mai_2017.jpg'

# 1) En base a lo visto en el paper ¿Qué significan los parámetros definidos en la siguiente celda?

*Respuesta:*

En la página 12 del paper se especifica la siguiente ecuación de Loss que luego se busca minimizar, la misma tiene 2 términos, uno correspondiente a la Loss de contenido y otro a la Loss de estilo:

    L_total(p,a,x) = α L_content(p,x) + β L_style(a,x)

Donde 'α' es el factor de peso que se le asigna a la Loss contenido y 'β' a la de estilo.

Aclaración: En esta implementación la variable (o hiperparámetro del modelo ) 'content_weight' se corresponde directamente con el 'α' de la ecuación ya que multiplica directamente a la Loss de contenido. Por otro lado, 'style_weight' no representa directamente el 'β' sino una parte, ya que en este caso el valor se divide por la cantidad de capas antes de multiplicarse por la Loss de estilo.
total_variation_weight es el valor que luego multiplica el total_variation_loss() de la imagen generada, como se verá más adelante.
Al no ser parámetros entrenables, asumimos que todos son hiperparámetros del modelo.

In [9]:
total_variation_weight = 0.1
style_weight = 10
content_weight = 1

In [10]:
total_variation_weight_calc = content_weight / style_weight
print(total_variation_weight_calc)

0.1


In [11]:
# Definimos el tamaño de las imágenes a utilizar
width, height = load_img(base_image_path).size
img_nrows = 400
img_ncols = int(width * img_nrows / height)
print(img_nrows, img_ncols)

400 517


In [12]:
base_image_path

PosixPath('/Users/hordia/dev/ML/course-ITBA-DL/course-notebooks-labs/TP-Final-Computer-Vision/hordia-Trabajo-Final-CV/content/775px-Neckarfront_Tübingen_Mai_2017.jpg')

In [13]:
style_reference_image_path

PosixPath('/Users/hordia/dev/ML/course-ITBA-DL/course-notebooks-labs/TP-Final-Computer-Vision/hordia-Trabajo-Final-CV/content/La_noche_estrellada1.jpg')

In [14]:
# funciones keras: from keras.preprocessing.image import load_img, save_img, img_to_array
img = load_img(base_image_path, target_size=(img_nrows, img_ncols))
print(img.size)

(517, 400)


In [15]:
img = img_to_array(img)
print(img.shape)

(400, 517, 3)


In [16]:
img = np.expand_dims(img, axis=0)
print(img.shape)

(1, 400, 517, 3)


# 2) Explicar qué hace la siguiente celda. En especial las últimas dos líneas de la función antes del return. ¿Por qué? Ayuda: https://keras.io/applications/

*Respuesta:* Es una función que se encarga de realizar todos los procesamientos que son necesarios para poder utilizar cualquier imagen con el modelo definido en esta notebook.

    img = load_img(image_path, target_size=(img_nrows, img_ncols))
    
La primer línea, a partir de un path y nombre de archivo, carga la imagen en una variable "img" y la redimensiona según los valores filas y columnas especificados en la celda anterior (el tamaño de las imagenes a utilizar)
    
    img = img_to_array(img)

Convierte la imagen objeto tipo PIL a un array numpy con el siguiente shape (400, 517, 3)

    np.expand_dims(img, axis=0)
    
Agrega al principio del array una dimensión extra, esto es porque muchos modelos de Deep Learning (incluyendo el VGG19) esperan un batch de imagenes como entrada. Con esta línea, se crea efectivamente un batch de una sola línea. El shape queda (1, 400, 517, 3)

    img = vgg19.preprocess_input(img)

La última línea es una función de Keras y termina de adaptar el array imagen al modelo con los pesos pre-entrenados VGG19. Convierte de el estándar para imagenes a color RGB (rojo, verde y azul) al BGR (azul, verde, rojo). Esta diferencia tiene que ver con el orden en que algunas bibliotecas para procesamiento tratan las imagenes a color. Además de esto, se centran los valores con respecto a la media del dataset ImageNet de referencia, pero sin escalar los mismos.

In [17]:
def preprocess_image(image_path):
    img = load_img(image_path, target_size=(img_nrows, img_ncols))
    img = img_to_array(img)
    img = np.expand_dims(img, axis=0)
    img = vgg19.preprocess_input(img)
    return img

# 3) Habiendo comprendido lo que hace la celda anterior, explique de manera muy concisa qué hace la siguiente celda. ¿Qué relación tiene con la celda anterior?

*Respuesta:*

Es una función para realizar el procesamiento "inverso" a lo definido en la celda anterior. Es decir, remueve el centrado a cero (sumando la media que antes se habia restado) y vuelve al formato RGB (el que espera por ejemplo matplotlib). Este paso es necesario para que no ocurra una interpretación incorrecta de los colores. Además, en la función también se encarga de asegurar que los valores se encuentran entre enteros 0-255 y con el mismo shape que las imagenes de entrada.

In [18]:
def deprocess_image(x):
    x = x.reshape((img_nrows, img_ncols, 3))
    # Remove zero-center by mean pixel
    x[:, :, 0] += 103.939
    x[:, :, 1] += 116.779
    x[:, :, 2] += 123.68
    # 'BGR'->'RGB'
    x = x[:, :, ::-1]
    x = np.clip(x, 0, 255).astype('uint8')
    return x

In [19]:
base_image_path.name

'775px-Neckarfront_Tübingen_Mai_2017.jpg'

In [20]:
style_reference_image_path.name

'La_noche_estrellada1.jpg'

In [21]:
# get tensor representations of our images
# K.variable convierte un numpy array en un tensor, para 
base_image = K.variable(preprocess_image(base_image_path))
style_reference_image = K.variable(preprocess_image(style_reference_image_path))

In [22]:
base_image.shape

TensorShape([1, 400, 517, 3])

In [23]:
%skip
import tensorflow as tf
tmp_img = tf.convert_to_tensor(img_to_array( load_img(base_image_path, target_size=(img_nrows, img_ncols)) ))
print(tmp_img.shape)
tmp_base_img = K.permute_dimensions(tmp_img, (2, 0, 1))
print(tmp_base_img.shape)

UsageError: Line magic function `%skip` not found.


In [24]:
combination_image = K.placeholder((1, img_nrows, img_ncols, 3))

Aclaración:

La siguiente celda sirve para procesar las tres imagenes (contenido, estilo y salida) en un solo batch.

In [25]:
# combine the 3 images into a single Keras tensor
input_tensor = K.concatenate([base_image,
                              style_reference_image,
                              combination_image], axis=0)

In [26]:
# build the VGG19 network with our 3 images as input
# the model will be loaded with pre-trained ImageNet weights
model = vgg19.VGG19(input_tensor=input_tensor,
                    weights='imagenet', include_top=False)
print('Model loaded.')

# get the symbolic outputs of each "key" layer (we gave them unique names).
outputs_dict = dict([(layer.name, layer.output) for layer in model.layers])

2023-08-28 00:34:52.171155: I metal_plugin/src/device/metal_device.cc:1154] Metal device set to: Apple M2 Pro
2023-08-28 00:34:52.171179: I metal_plugin/src/device/metal_device.cc:296] systemMemory: 16.00 GB
2023-08-28 00:34:52.171185: I metal_plugin/src/device/metal_device.cc:313] maxCacheSize: 5.33 GB
2023-08-28 00:34:52.171682: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:303] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2023-08-28 00:34:52.171705: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:269] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)
2023-08-28 00:34:52.181139: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:375] MLIR V1 optimization pass is not enabled
2023-08-28 00:34:52.187544: I tensorflow/core/gra

Model loaded.


2023-08-28 00:34:52.518978: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


# 4) En la siguientes celdas:

- *¿Qué es la matriz de Gram?¿Para qué se usa?*

Se utiliza para capturar la información de estilo de una imagen. Se obtiene realizando la correlación de los features maps para cada capa (layer). Cada elemento de la matriz representa la correlación entre las "filter responses" (respuestas al filtro) en 2 posiciones espaciales diferentes. Esta correlación, de alguna forma codifica información sobre como los diferentes features interactuan entre sí y por lo tanto se asume que representan la información de estilo.

$$ G^l_{ij} = \sum{k} F^l_{ik} F^l_{jk} $$

- *¿Por qué se permutan las dimensiones de x?*

Recordemos que con el preprocesamiento el shape del vector imagen tenia la forma (alto, ancho, canales), con esta permutación se cambia el orden de las dimensiones y queda (canales, ancho, alto) para luego aplicar el batch_flatten que convierte a 2D manteniendo la primer dimensión. Con los vectores que representan la imagen, se puede calcular el producto punto para terminar de calcular la matriz de Gram.


In [27]:
def gram_matrix(x):
    features = K.batch_flatten(K.permute_dimensions(x, (2, 0, 1)))
    gram = K.dot(features, K.transpose(features))
    return gram

# 5) Losses:

Explicar qué mide cada una de las losses en las siguientes tres celdas.

*Rta:*

La función style_loss() calcula la Loss de estilo, mediante el cálculo de las matrices de Gram que representan los features maps de estilo de cada imagen, por lo tanto al tomar diferencia como función de Loss que se intentará minimizar, se buscará mantener el estilo contra la imagen de referencia.
Responde a la ecuación (4) y (5) del paper 
$$ E_l = \frac{1}{4N_l^2 M_l^2} \sum_{i,j} (G^l_{ij} − A^l_{ij})^2 $$ 
(E_l, para cada layer)
$$ L_{estilo}(a,x) = \sum_{l=0}^L w_l E_l $$


La función content_loss() calcula la Loss para el contenido, la ecuación (1) que es el error cuadrático medio entre las dos representaciones de features
$$ L_{contenido} = \frac{1}{2} \sum_{i,j} (F^l_{ij} − P^l_{ij})^2 $$


Por último total_variation_loss() mide la variación de la imagen de entrada, tomando diferencias absolutas entre valores vecinos, lo pondera por un factor predeterminado (1.25) y lo suma. Esto fomenta reducir el ruido de alta frecuencia en las imágenes generadas, es decir contar con transiciones más suaves.

In [28]:
def style_loss(style, combination):
    assert K.ndim(style) == 3
    assert K.ndim(combination) == 3
    S = gram_matrix(style)
    C = gram_matrix(combination)
    channels = 3
    size = img_nrows * img_ncols
    return K.sum(K.square(S - C)) / (4.0 * (channels ** 2) * (size ** 2))

In [29]:
def content_loss(base, combination):
    return K.sum(K.square(combination - base))


In [30]:
# Se define el 1.25 como constante (hyperparámetro)
def total_variation_loss(x):
    assert K.ndim(x) == 4
    a = K.square(
        x[:, :img_nrows - 1, :img_ncols - 1, :] - x[:, 1:, :img_ncols - 1, :])
    b = K.square(
        x[:, :img_nrows - 1, :img_ncols - 1, :] - x[:, :img_nrows - 1, 1:, :])
    return K.sum(K.pow(a + b, 1.25))


In [31]:
# RECORDAR: combine the 3 images into a single Keras tensor
# input_tensor = K.concatenate([base_image, style_reference_image, combination_image], axis=0)
# TENSOR
#   0 --> BASE IMG
#   1 --> STYLE REFERENCE IMG
#   2 --> COMBINATION IMG

In [32]:
REF_BASE_IMG = 0
REF_STYLE_IMG = 1
REF_COMB_IMG = 2

In [33]:
# Armamos la loss total
loss = K.variable(0.0)

layer_features = outputs_dict['block5_conv2']

base_image_features = layer_features[REF_BASE_IMG, :, :, :]
combination_features = layer_features[REF_COMB_IMG, :, :, :]

loss = loss + content_weight * content_loss(base_image_features,combination_features)

feature_layers = ['block1_conv1', 'block2_conv1',
                  'block3_conv1', 'block4_conv1',
                  'block5_conv1']

for layer_name in feature_layers:
    layer_features = outputs_dict[layer_name]
    style_reference_features = layer_features[REF_STYLE_IMG, :, :, :] 
    combination_features = layer_features[REF_COMB_IMG, :, :, :]
    
    sl = style_loss(style_reference_features, combination_features)
    loss = loss + (style_weight / len(feature_layers)) * sl

loss = loss + total_variation_weight * total_variation_loss(combination_image)

In [34]:
grads = K.gradients(loss, combination_image)

outputs = [loss]
if isinstance(grads, (list, tuple)):
    outputs += grads
else:
    outputs.append(grads)

f_outputs = K.function([combination_image], outputs)

# 6) Explique el propósito de las siguientes tres celdas. ¿Qué hace la función fmin_l_bfgs_b? ¿En qué se diferencia con la implementación del paper? ¿Se puede utilizar alguna alternativa?

*Respuesta:*

*fmin_l_bfgs_b()* es una función de la biblioteca SciPy. Se trata de una implementación de la optimización de Broyden-Fletcher-Goldfarb-Shanno (L-BFGS-B) en particular, de un algoritmo de optimización diseñado para memoria limitada (ver paper en la referencia). Es una técnica de optimización numérica que se utiliza para encontrar los mínimos (óptimos) de una función objetivo en función de varias variables, sujetas a restricciones de límites en esas variables.

Ref: https://docs.scipy.org/doc/scipy/reference/generated/scipy.optimize.fmin_l_bfgs_b.html 

El algoritmo L-BFGS-B es adecuado para problemas de optimización sin restricciones o con restricciones con límites (es decir, cuando las variables deben mantenerse dentro de ciertos rangos). A diferencia de algunos otros algoritmos de optimización, L-BFGS-B no requiere cálculos de derivadas y es eficiente en problemas donde el número de variables es grande.

En las siguiente celda se define una clase Evaluator con tres métodos. Uno default de incialización, uno de loss y otro de gradientes. El primero ejecuta la función eval_loss_and_grads(x) que retorna loss_value y grad_value. El método grads(), realiza una copia. Esto se utiliza en el punto 7)

En el paper utilizan el método de Gradientes Descendientes para la minimización, que ajusta los parámetros en la dirección opuesta al gradiente de la función objetivo para minimizarla. Es el de los métodos de minimización más simple y es el que se suele utilizar habitualmente. Tiene como hyperparámetro el Learning Rate y utiliza derivadas. L-BFGS-B es una técnica más avanzada que a menudo converge más rápido y es más robusta en problemas de optimización convexa.




In [35]:
def eval_loss_and_grads(x):
    # Reshape the input image to match the expected input shape for the model
    x = x.reshape((1, img_nrows, img_ncols, 3))
    # Compute the loss and gradients for the input image
    outs = f_outputs([x])
    # Extract the loss value from the computed outputs
    loss_value = outs[0]
    # If there is only one gradient value, flatten it and convert to float64
    if len(outs[1:]) == 1:
        grad_values = outs[1].flatten().astype('float64')
    # If there are multiple gradient values, convert them to a flat numpy array of float64
    else:
        grad_values = np.array(outs[1:]).flatten().astype('float64')
    # Return the computed loss and gradient values
    return loss_value, grad_values

# this Evaluator class makes it possible
# to compute loss and gradients in one pass
# while retrieving them via two separate functions,
# "loss" and "grads". This is done because scipy.optimize
# requires separate functions for loss and gradients,
# but computing them separately would be inefficient.

In [36]:
class Evaluator(object):

    def __init__(self):
        self.loss_value = None
        self.grads_values = None

    def loss(self, x):
        assert self.loss_value is None
        loss_value, grad_values = eval_loss_and_grads(x)
        self.loss_value = loss_value
        self.grad_values = grad_values
        return self.loss_value

    def grads(self, x):
        assert self.loss_value is not None
        grad_values = np.copy(self.grad_values)
        self.loss_value = None
        self.grad_values = None
        return grad_values

# 7) Ejecute la siguiente celda y observe las imágenes de salida en cada iteración.

In [37]:
!mkdir -p content/output

In [38]:
print(base_image_path.name)
print(style_reference_image_path.name)

775px-Neckarfront_Tübingen_Mai_2017.jpg
La_noche_estrellada1.jpg


In [39]:
total_variation_weight, style_weight, content_weight

(0.1, 10, 1)

In [40]:

print("Iterations: ", iterations)

Iterations:  100


In [ ]:
evaluator = Evaluator() # 

# run scipy-based optimization (L-BFGS) over the pixels of the generated image
# so as to minimize the neural style loss
x = preprocess_image(base_image_path)

for i in range(iterations):
    print('Start of iteration', i)
    start_time = time.time()
    x, min_val, info = fmin_l_bfgs_b(evaluator.loss, x.flatten(),
                                     fprime=evaluator.grads, maxfun=20)
    print('Current loss value:', min_val)
    # save current generated image
    img = deprocess_image(x.copy())
    fname = result_prefix / ('output_at_iteration_%d.png' % i)
    save_img(fname, img)
    end_time = time.time()
    print('Image saved as', fname)
    print('Iteration %d completed in %ds' % (i, end_time - start_time))

# 8) Generar imágenes para distintas combinaciones de pesos de las losses. Explicar las diferencias. (Adjuntar las imágenes generadas como archivos separados.)

*Respuesta:*
En la siguientes celda se modifican los hiperparámetros con distintas combinaciones de pesos para obtener diferentes resultados.

### Referencia output punto 7

<center>

| Content | Style |
| --- | --- |
| <img src='./content/775px-Neckarfront_Tübingen_Mai_2017.jpg'> | <img src='./content/La_noche_estrellada1.jpg'> |


| total_variation_weight | style_weight | content_weight | interations |
| --- | --- | --- | --- |
| 0.01 | 10 | 1 | 100 |



<img src='./resultados/PUNTO-7/output_at_iteration_99.png'>

</center>

In [42]:
# G
# Combinación 1
total_variation_weight = 0.01
style_weight = 50
content_weight = 1

iterations = 100

<center>

| total_variation_weight | style_weight | content_weight | interations |
| --- | --- | --- | --- |
| 0.01 | 50 | 1 | 100 |

<img src='./resultados/PUNTO-8/output_Neckarfront_NOCHE-ESTRELLADA_0.01_50_1_at_iteration_99.png'>

*Obs: .*

In [43]:
# Genero contenido más parecido al original cambiando los pesos
# Combinación 2
total_variation_weight = 0.01
style_weight = 1
content_weight = 1000

iterations = 100

<center>

| total_variation_weight | style_weight | content_weight | interations |
| --- | --- | --- | --- |
| 0.01 | 1 | 1000 | 100 |

<img src='./resultados/PUNTO-8/output_Neckarfront_NOCHE-ESTRELLADA_0.01_1_1000_at_iteration_99.png'>

*Obs: Al aumentar el peso del contenido y bajar el total variation weight se observa, como era de esperar, un leve cambio de estilo, manteniendo el foto realismo.*


# 9) Cambiar las imágenes de contenido y estilo por unas elegidas por usted. Adjuntar el resultado.

Respuesta:

In [44]:
# Definimos las imagenes que vamos a utilizar, y el directorio de salida

# Base
base_image_path = Path.cwd() / Path("content-propio/Toto_small.jpg")

# Style
# style_reference_image_path = Path.cwd() / Path("content/La_noche_estrellada1.jpg")

# style_reference_image_path = Path.cwd() / Path("content-propio/nieve_small.jpg")
style_reference_image_path = Path.cwd() / Path("content-propio/cubism.png")


result_prefix = Path.cwd() / Path("content-propio/output")


In [45]:
print(base_image_path.name)
print(style_reference_image_path.name)

Toto_small.jpg
cubism.png


In [46]:
! mkdir -p content-propio/output

In [47]:
# Definimos el tamaño de las imágenes a utilizar
width, height = load_img(base_image_path).size
# img_nrows = 400
# img_ncols = int(width * img_nrows / height)
img_nrows = height
# img_ncols = int(width * img_nrows / height)
img_ncols = width
print(img_nrows, img_ncols)

400 700


In [48]:
# total_variation_weight = 0.01
total_variation_weight = 0.5
# total_variation_weight = 0.1

# style_weight = 1000
style_weight = 10

content_weight = 1

iterations = 100
# iterations = 300
# iterations = 3
# iterations = 10

<center>

| Content | Style |
| --- | --- |
| <img src='./content-propio/Toto_small.jpg'> | <img src='./content/La_noche_estrellada1.jpg'> |


| total_variation_weight | style_weight | content_weight | interations |
| --- | --- | --- | --- |
| 0.01 | 10 | 1 | 100 |


<img src='./resultados/PUNTO-9/output_TOTO_NOCHE-ESTRELLADA_0.1_10_1_at_iteration_99.png'>

</center>


<center>

| Content | Style |
| --- | --- |
| <img src='./content-propio/Toto_small.jpg'> | <img src='./content-propio/nieve_small.jpg'> |


| total_variation_weight | style_weight | content_weight | interations |
| --- | --- | --- | --- |
| 0.01 | 10 | 1 | 100 |


<img src='./resultados/PUNTO-9/output_TOTO_Nieve_0.1_10_1_at_iteration_99.png'>

</center>

Obs: 

<center>

| Content | Style |
| --- | --- |
| <img src='./content-propio/Toto_small.jpg'> | <img src='./content-propio/kandinsky_small.jpg'> |


| total_variation_weight | style_weight | content_weight | interations |
| --- | --- | --- | --- |
| 0.01 | 10 | 1 | 40 |


<img src='./resultados/PUNTO-9/output_TOTO_Kandinsky_0.01_10_1_at_iteration_40.png'>

</center>

Obs: 

In [49]:
total_variation_weight, style_weight, content_weight

(0.5, 10, 1)

<center>

| Content | Style |
| --- | --- |
| <img src='./content-propio/Toto_small.jpg'> | <img src='./content-propio/cubism.png'> |


| total_variation_weight | style_weight | content_weight | interations |
| --- | --- | --- | --- |
| 0.5 | 10 | 1 | 100 |


<img src='./resultados/PUNTO-9/output_TOTO_Cubism_0.5_10_1_at_iteration_99.png'>

</center>
